In [9]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import datetime
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy
import sys,os

sys.path.append('../src')
import utils as ut
import warnings
warnings.filterwarnings("ignore")

In [10]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf']
ds.close()

In [14]:
def get_diag(run,vname,region,verbose=False):
    assert region in ['Alaska','Antarctic','Global']
    fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*T.nc'))
    if run == 'n011':
        fnames = fnames[38:]
          
    nmonths = 0
    for fname in fnames:
        ds = xr.open_dataset(fname)
        nmonths += len(ds.time_counter)
        ds.close() 
    
    time = np.zeros((nmonths))
    varav = np.zeros((nmonths))
    varmax = np.zeros((nmonths))
    varmin = np.zeros((nmonths))

    c = 0
    for f,fname in enumerate(fnames):
        ds = xr.open_dataset(fname)
        darea = area.copy()
        if region == 'Alaska':
            ds = ds.isel(y=slice(250,-10),x=slice(120,150))
            darea = darea.isel(y_3=slice(250,-10),x_3=slice(120,150))
        elif region == 'Antarctic':
            ds = ds.isel(y=slice(0,45))
            darea = darea.isel(y_3=slice(0,45))
        var = ds[vname]
        for t,tt in enumerate(ds.time_counter):
            time[c] = c/12
            vvar = var.isel(time_counter=t)
            try:
                vvar = vvar.isel(olevel=34)
            except:
                pass
            vvar = np.where(vvar==0,np.nan,vvar)
            ddarea = np.where(np.isnan(vvar),np.nan,darea)
            varav[c] = np.nansum(vvar*ddarea)/np.nansum(ddarea)
            varmax[c] = np.nanmax(vvar)
            varmin[c] = np.nanmin(vvar)
            #print(time[c],thickav[c],thickmax[c])
            c+=1
        ds.close()
        if verbose: print(run,region,vname,fname)
        
    varav2  = xr.DataArray(varav,dims=('time'),coords={'time':time})
    varmax2 = xr.DataArray(varmax,dims=('time'),coords={'time':time})
    varmin2 = xr.DataArray(varmin,dims=('time'),coords={'time':time})

    if run=='n011':
        rrun = 'ctrl'
    else:
        rrun = run
    ds = xr.Dataset({'mean':varav2,'max':varmax2,'min':varmin2})
    savename = f'../data/diag_{rrun}_{region}_{vname}.nc'
    ds.to_netcdf(savename,mode='w')
    ds.close()
        
    print('Saved',savename)
    return

In [15]:
runs = ['hal2']
#runs = ['n011','quar','half','hal2','totl']

regions = ['Alaska','Antarctic','Global']
vnames = ['zos','sos','tos','so','thetao','vfxice','friver','calving']

for region in regions:
    for run in runs:
        for vname in vnames:
            get_diag(run,vname,region)
print('Finished')

Saved ../data/diag_hal2_Alaska_zos.nc
Saved ../data/diag_hal2_Alaska_sos.nc
Saved ../data/diag_hal2_Alaska_tos.nc
Saved ../data/diag_hal2_Alaska_so.nc
Saved ../data/diag_hal2_Alaska_thetao.nc
Saved ../data/diag_hal2_Alaska_vfxice.nc
Saved ../data/diag_hal2_Alaska_friver.nc
Saved ../data/diag_hal2_Alaska_calving.nc
Saved ../data/diag_hal2_Antarctic_zos.nc
Saved ../data/diag_hal2_Antarctic_sos.nc
Saved ../data/diag_hal2_Antarctic_tos.nc
Saved ../data/diag_hal2_Antarctic_so.nc
Saved ../data/diag_hal2_Antarctic_thetao.nc
Saved ../data/diag_hal2_Antarctic_vfxice.nc
Saved ../data/diag_hal2_Antarctic_friver.nc
Saved ../data/diag_hal2_Antarctic_calving.nc
Saved ../data/diag_hal2_Global_zos.nc
Saved ../data/diag_hal2_Global_sos.nc
Saved ../data/diag_hal2_Global_tos.nc
Saved ../data/diag_hal2_Global_so.nc
Saved ../data/diag_hal2_Global_thetao.nc
Saved ../data/diag_hal2_Global_vfxice.nc
Saved ../data/diag_hal2_Global_friver.nc
Saved ../data/diag_hal2_Global_calving.nc
